In [ ]:
#Import libraries
import pandas as pd
import numpy as np

In [3]:
#Replacing Nan standardizes them as missing data

df = df.replace([np.inf, -np.inf], np.nan)

In [4]:
#Generate summary statistics for each numeric column

df.describe()

,case,year,month,Weights_QTR,Air_Terminal,Length of Stay,shopping_fash,shopping_jewllery,shopping_watches,shopping_wellness,...,Travel companion - Alone,Travel companion - Spouse,Travel companion - Your Child/Children,Travel companion - Parents/Parents-in-law,Travel companion - Grandparents/Grandparents-in-law,Travel companion - Siblings,Travel companion - Other relatives,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others
count,22972.000000,22974.0,22974.000000,22974.000000,17977.000000,22974.000000,22889.000000,22974.000000,22974.000000,22974.000000,...,22974.000000,22974.000000,22974.000000,22974.000000,22974.000000,22974.000000,22974.000000,22974.000000,22974.000000,22974.000000
mean,11496.938186,2018.0,6.497301,634.074916,1.964566,3.418821,107.073232,12.998617,9.918191,44.889603,...,0.248237,0.432184,0.108514,0.063376,0.003265,0.044833,0.021764,0.148298,0.041656,0.010185
std,6632.497501,0.0,3.460326,600.404641,0.844444,3.310353,248.154460,125.419017,91.056213,104.936418,...,0.432000,0.495390,0.311035,0.243643,0.057044,0.206942,0.145914,0.355403,0.199806,0.100410
min,1.000000,2018.0,1.000000,31.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5754.750000,2018.0,4.000000,418.326516,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,11497.500000,2018.0,7.000000,516.828571,2.000000,3.000000,8.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,17240.250000,2018.0,9.000000,692.625000,3.000000,4.000000,125.000000,0.000000,0.000000,50.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,22983.000000,2018.0,12.000000,14673.000000,3.000000,60.000000,6000.000000,8000.000000,5244.500000,2000.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
#Performed data validation check by comparing the summed shopping components with the reported total expenditure and identified discrepancies between the two

df['total_shopping'] = df['totacc'] + df['totfnb'] + df['tottran'] + df['totbiz'] + df['totedu'] + df['totmedi'] + df['tototh'] + df['totshopping_rep']
df['total_diff_shopping_totexp'] = np.isclose(df['total_shopping'], df['tot.exp'], atol=0.01)
df['total_diff_shopping_totexp'].value_counts()

total_diff_shopping_totexp
True     22869
False      105
Name: count, dtype: int64

In [6]:
#Performed a data validation check and identified cases where the stated main purpose of visit did not logically align with the total expenditure 

df[['Main Purpose of Visit', 'tot.exp']].sort_values(by='tot.exp', ascending=False)

,Main Purpose of Visit,tot.exp
13910,General business purpose,29206.42
9860,Sightseeing/ Attractions,23791.22
2929,Accompanying a healthcare/ medical visitor fo...,15977.60
6392,Accompanying a healthcare/ medical visitor fo...,15218.74
1376,Corporate/ business meetings (a. Venue of co...,11474.71
...,...,...
14817,Visiting friends/ relatives (who are not inter...,0.00
14816,Holiday/ Rest & Relax,0.00
14814,Visiting friends/ relatives (who are not inter...,0.00
20519,Holiday/ Rest & Relax,0.00


High expenditure observations are predominately associated with business related purpose of visit, indicating that business travelers can incur substantially higher total spending compared to leisure oriented visitors

In [7]:
#Created expenditure based percentile segments to categorize visitors into distinct spending groups

p25 = df['tot.exp'].quantile(0.25)
p75 = df['tot.exp'].quantile(0.75)
p95 = df['tot.exp'].quantile(0.95)

def spend_segment(x):
    if x <= p25:
        return "Bottom 25% (Budget)"
    elif x <= p75:
        return "25% - 75% (Mainstream)"
    elif x <= p95:
        return "75% - 95% (Affluent)"
    else:
        return "Top 5% (Premium)"

df["spend_group"] = df['tot.exp'].apply(spend_segment)
df['spend_group'].value_counts()

spend_group
25% - 75% (Mainstream)    11486
Bottom 25% (Budget)        5744
75% - 95% (Affluent)       4595
Top 5% (Premium)           1149
Name: count, dtype: int64

In [8]:
#Calculated daily spending and derived the average spend per day for each expenditure segment

los = df['Length of Stay'].replace(0, np.nan)
df['spend_per_day'] = (df['tot.exp'] / los).round(2)
df['avg_spend_segment'] = (df.groupby('spend_group', dropna=False)['spend_per_day'].transform('mean').round(2))
df.groupby('spend_group')['avg_spend_segment'].value_counts()

spend_group             avg_spend_segment
25% - 75% (Mainstream)  309.73               11486
75% - 95% (Affluent)    580.11                4595
Bottom 25% (Budget)     53.81                 5744
Top 5% (Premium)        1351.84               1149
Name: count, dtype: int64

In [9]:
#Performed an analysis on Great Singapore Sale during June Period

df_analysis = df[['month', 'Purpose of Visit', 'Main Purpose of Visit', 'Occupation', 'totshopping_rep', 'tot.exp', 'spend_per_day', 'spend_group']]
df_6 = df_analysis[df_analysis['month'] == 6]
df_6[df_6['Main Purpose of Visit'].str.contains('To shop', regex=True, na=False)]
df_6['spend_group'].value_counts()

spend_group
25% - 75% (Mainstream)    844
Bottom 25% (Budget)       459
75% - 95% (Affluent)      393
Top 5% (Premium)           92
Name: count, dtype: int64

In [10]:
#Performed an analysis on F1 Singapore between September & October

df_9_10 = df_analysis[df_analysis['month'].isin([9, 10])]
df_9_10[['month', 'Purpose of Visit', 'Main Purpose of Visit', 'Occupation', 'totshopping_rep', 'tot.exp', 'spend_per_day', 'spend_group']].sort_values(by='tot.exp', ascending=False)
df_9_10['spend_group'].value_counts()

spend_group
25% - 75% (Mainstream)    1910
75% - 95% (Affluent)       790
Bottom 25% (Budget)        769
Top 5% (Premium)           189
Name: count, dtype: int64

In [11]:
#Categorized travellers into companion based groups such as solo, business, couple, family, friends, and others

def travel_companion(x):
    if x['Travel companion - Alone'] == 1:
        return 'Solo'
    elif x['Travel companion - Business associates/Colleagues'] == 1:
        return 'Business Trips'
    elif x['Travel companion - Spouse'] == 1:
        return 'Couple'
    elif x['Travel companion - Your Child/Children'] == 1 or x['Travel companion - Parents/Parents-in-law'] == 1 or x['Travel companion - Grandparents/Grandparents-in-law'] == 1 or x['Travel companion - Siblings'] == 1 or x['Travel companion - Other relatives'] == 1:
        return 'Family'
    elif x['Travel companion - Friends'] == 1:
        return 'Friends'
    else:
        return 'Mixed & Others'

df['travel_groups'] = df.apply(travel_companion, axis=1)
df['travel_groups'].value_counts()

travel_groups
Couple            9895
Solo              5703
Friends           2768
Family            2135
Mixed & Others    1516
Business Trips     957
Name: count, dtype: int64

In [15]:
#Identified hotel stays by validating accommodation data and creating a binary indicator based on the presence of a main hotel as c4a_1 did not accurately reflect hotel stays

df['hotel_stay'] = df['MainHotel'].notna().astype(int)
df

,case,year,month,Country of Residence,City of Residence,Purpose of Visit,Main Purpose of Visit,Weights_QTR,Air_Terminal,Sea_Terminal,...,Travel companion - Friends,Travel companion - Business associates/Colleagues,Travel companion - Others,total_shopping,total_diff_shopping_totexp,spend_group,spend_per_day,avg_spend_segment,travel_groups,hotel_stay
0,1.0,2018,1,India,Delhi,Leisure,Holiday/ Rest & Relax,660.046512,2.0,NaN,...,0,0,0,1480.90,True,75% - 95% (Affluent),246.82,580.11,Couple,1
1,2.0,2018,1,India,Delhi,Leisure,Visiting friends/ relatives (who are not inter...,433.217949,2.0,NaN,...,0,0,0,99.10,True,Bottom 25% (Budget),1.98,53.81,Solo,0
2,3.0,2018,1,United Arab Emirates,Dubai,Leisure,General business purpose,335.125000,2.0,NaN,...,0,0,0,5286.27,True,Top 5% (Premium),881.04,1351.84,Solo,0
3,4.0,2018,1,India,Others,Leisure,Holiday/ Rest & Relax,490.977273,2.0,NaN,...,0,0,0,674.50,True,25% - 75% (Mainstream),96.36,309.73,Couple,0
4,5.0,2018,1,Canada,Toronto,Business + Accompanying Pax,Visiting an international student (relative or...,379.800000,2.0,NaN,...,0,0,0,4182.55,True,Top 5% (Premium),2091.27,1351.84,Solo,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22969,7789.0,2018,12,Australia,Brisbane,Leisure,Holiday/ Rest & Relax,310.354839,2.0,NaN,...,0,0,0,1399.88,True,75% - 95% (Affluent),349.97,580.11,Couple,1
22970,7790.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,109.393939,2.0,NaN,...,1,0,0,548.53,True,25% - 75% (Mainstream),182.84,309.73,Friends,1
22971,7791.0,2018,12,Hong Kong,Hong Kong,Leisure,Holiday/ Rest & Relax,326.906250,2.0,NaN,...,0,0,0,454.31,True,25% - 75% (Mainstream),64.90,309.73,Couple,1
22972,7792.0,2018,12,Hong Kong,Hong Kong,Business + Accompanying Pax,Gather information/ facts on the education ser...,820.615385,2.0,NaN,...,0,0,0,297.42,True,Bottom 25% (Budget),297.42,53.81,Solo,0


In [13]:
#Analyzed relationship between travel companion groups and hotel stay behaviour

df.value_counts(['travel_groups', 'hotel_stay']).unstack(fill_value=0)

hotel_stay,0,1
travel_groups,,
Business Trips,185,772
Couple,2078,7817
Family,840,1295
Friends,673,2095
Mixed & Others,338,1178
Solo,2576,3127


In [14]:
#Analyzed relationship between country of residence and spend group (Top 5% Premium) and average length of stay

df_nation = df.value_counts(['Country of Residence', 'spend_group']).unstack(fill_value=0)
los_nation = (df.groupby('Country of Residence')['Length of Stay'].mean().round(2))
df_nation_stay = df_nation.join(los_nation)
df_nation_stay.sort_values(by='Top 5% (Premium)', ascending=False)

,25% - 75% (Mainstream),75% - 95% (Affluent),Bottom 25% (Budget),Top 5% (Premium),Length of Stay
Country of Residence,,,,,
India,523,408,337,183,4.13
China,1545,743,503,169,3.55
Indonesia,1946,517,1590,168,2.74
United States,382,176,144,59,3.65
Japan,644,276,215,54,3.65
...,...,...,...,...,...
Nepal,10,3,3,0,3.88
Nigeria,3,1,0,0,8.50
North Korea,2,0,0,0,3.50
